# ``Dribble``'s Command Line Interface

``Dribble`` is a library providing tools for analyzing diffusion pathways in crystalline materials with substitutional disorder.  However, some of this functionality can be accessed using the included command line tool which is also called ``dribble``.

The ``dribble`` command line tool requires two input files

- The general ``dribble`` input file in ``JSON`` format; and
- An atomic structure file in ``VASP``'s ``POSCAR`` format.

## Application: Site Percolation on the FCC Lattice

As a simple example with known solution, let's consider the site percolation problem on the FCC lattice.  You can read more about [*percolation theory* on Wikipedia](https://en.wikipedia.org/wiki/Percolation_theory).  The site percolation threshold for the FCC structure is known from published numerical simulations and is about $x_c \approx 0.199$ [according to Wikipedia](https://en.wikipedia.org/wiki/Percolation_threshold#Thresholds_on_3D_lattices).

The site percolation problem asks the following question: Given a randomly occupied lattice, what is the lowest concentration of occupied lattice sites needed so that a percolating network of nearest-neighbor bonds is present.  This concentration is called **percolation threshold**.  Note that in this basic example, two neighboring sites are considered bonded when both are occupied.  This **bond rule** is implemented in ``dribble`` and is named ``NearestNeighborBR``.

## General Input File

Let's create an example ``dribble`` input file:

In [1]:
%%writefile input-bond-rule.json
{
    "structure": "../LiMO2-Crystal-Structures/Cu-fcc.vasp",
    "formula_units": 1.0,
    "sublattices": {
        "A": {
            "description": "Copper sites",
            "sites": {"species": ["Cu"]},
            "initial_occupancy": {"Vac": 1.0}
        }
    },
    "bonds": [
        {
            "sublattices": ["A", "A"],
            "bond_rules": [["NearestNeighborBR"]]
        }
    ],
    "percolating_species": ["Cu"],
    "flip_sequence": [["Vac", "Cu"]]
}

Overwriting input-bond-rule.json


Let's go through the ``dribble`` input file section by section:

- The input atomic **structure** is the structure of FCC copper; 
- The number of **formula units** that the structure file corresponds to is 1;
- There is only one **sublattice**, and we name it "A".  This sublattice is identified by species, as all sites (there is only one) are occupied by Cu.  At the beginning of the simulation, (a fraction of 1.0 of) all sites shall be initialized as vacancies ("Vac" species);
- The only allowed **bonds** are between two sites of sublattice "A" following the ``NearestNeighborBR`` bond rule;
- The **percolating species** is Cu for no particular reason; and
- The simulation's **flip sequence** is such that all vacancy sites are one-by-one flipped to Cu.

## Structure Input File

The path to the atomic structure file is already defined in the general input file which points to the following POSCAR file:

In [2]:
%cat ../LiMO2-Crystal-Structures/Cu-fcc.vasp

FCC Structure
3.6
     0.0      0.5      0.5
     0.5      0.0      0.5
     0.5      0.5      0.0
Cu
1
direct
     0.0      0.0      0.0 Cu


**Note:** Instead of specifying the structure file in the general input file, we could have also used a command line argument.  This can be useful when applying the same simulation parameters to a number of different structure files.

## Calculation of the Percolation Threshold

We can now run the ``dribble`` command line tool to calculate the percolation threshold for the FCC site percolation problem.  We cannot just use the primitive unit cell to determine the percolation threshold.  For technical reasons, the cell size has to be at least large enough so that the second-nearest neighbors of a site are not periodic images of the site itself.  In addition, **the percolation threshold converges with the size of the cell size used for the Monte Carlo simulation**.  Hence, we have to try increasingly larger cells to make sure that the value has converged.

For the sake of time, we will consider a relatively small cell size in this example.

Run ``dribble`` on the command line with option `-p` to compute the percolation threshold.  The cell size relative to the input structure is specified with the `--supercell` flag.  Here we will use a $10\times{}10\times{}10$ supercell (i.e., 1000 sites) which is still relatively small:

In [8]:
! dribble input-bond-rule.json --supercell 10 10 10 -p


 Parsing input file 'input-bond-rule.json'... done.

 Setting up lattice and neighbor lists... done.

 Lattice and Sites
 -----------------

 Lattice vectors:

     0.00000000   18.00000000   18.00000000
    18.00000000    0.00000000   18.00000000
    18.00000000   18.00000000    0.00000000

 total number of sites : 1000
 occupied sites        : 0 (0 static)
 vacant sites          : 1000 (0 static)
 number of NNs         : 12

 Neighbor List
 -------------

 interaction range          : NNs only
 boxes per lattice direction: 5 5 5
 total number of atoms      : 1000
 av. number of atoms per box: 8.0

 Initializing percolator... done.

 MC percolation simulation
 -------------------------

 Calculating an estimate for the percolation point p_c.
 Initial composition: Vac 1.00 
 Averaging over 500 samples:

 0%                25%                 50%                 75%                 100%
 ▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮ done.

 Average pe

With the $10\times{}10\times{}10$ supercell, the percolation threshold (`p_c1` in the output) is $x_c > 0.2$ which is still slightly higher than the literature value of $0.199$ (though the accuracy might be sufficient for your application).  Please go ahead and try a $20\times{}20\times{}20$ supercell if you run a reasonably new computer.  On a 2015 laptop the calculation takes around 12 minutes.  The result will be closer to the converged value.

**Note:** The method used by `dribble` is a Monte Carlo (MC) algorithm, and therefore the precise numbers will differ from run to run.  The differences will become smaller with increasing cell size and increasing number of MC samples (the default is 500; can be adjusted with the `--samples` flag).

In [7]:
# uncomment the follwing line to run the 20x20x20 supercell
#! dribble input-bond-rule.json --supercell 20 20 20 -p


 Parsing input file 'input-bond-rule.json'... done.

 Setting up lattice and neighbor lists... done.

 Lattice and Sites
 -----------------

 Lattice vectors:

     0.00000000   36.00000000   36.00000000
    36.00000000    0.00000000   36.00000000
    36.00000000   36.00000000    0.00000000

 total number of sites : 8000
 occupied sites        : 0 (0 static)
 vacant sites          : 8000 (0 static)
 number of NNs         : 12

 Neighbor List
 -------------

 interaction range          : NNs only
 boxes per lattice direction: 9 9 9
 total number of atoms      : 8000
 av. number of atoms per box: 10.973936899862826

 Initializing percolator... done.

 MC percolation simulation
 -------------------------

 Calculating an estimate for the percolation point p_c.
 Initial composition: Vac 1.00 
 Averaging over 500 samples:

 0%                25%                 50%                 75%                 100%
 ▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮ don